# IF Analysis: Dual Channel Segmentation (Cell + Protein Markers)

This notebook segments both channels in immunofluorescence TIF images:
1. **Channel 1 (Cell Marker):** Large cells (200-300 pixels) using Cellpose
2. **Channel 2 (Protein Marker):** Small protein markers (10-20 pixels) using Cellpose

## 1. Setup and Imports

In [96]:
import numpy as np
import tifffile
from skimage import io
from cellpose import models
from cellpose import plot
import os
import torch
from pathlib import Path
import cv2
from scipy import ndimage
import napari

# Check for GPU
use_gpu = torch.cuda.is_available()
print(f"GPU Available: {use_gpu}")

GPU Available: True


## 2. Load Images

Set your folder path containing 2-channel TIF files or specify a single file.

In [ ]:


# Option 2: Process all TIF files in a folder (recommended for batch processing)
FOLDER_PATH = r"S:\micro\ts2625\eh2888\lem\20260128_EVE_LargeImage\tiff\quantification\test"  # Update this path

# Get all TIF files
tif_files = sorted(Path(FOLDER_PATH).glob("*.tif")) + sorted(Path(FOLDER_PATH).glob("*.tiff"))

print(f"Found {len(tif_files)} TIF files in {FOLDER_PATH}")
for i, f in enumerate(tif_files, 1):
    print(f"  {i}. {f.name}")


Found 1 TIF files in S:\micro\ts2625\eh2888\lem\20260128_EVE_LargeImage\tiff\quantification\test
  1. Saline_Gastro_0002_00001-12912-04334.tif

Loading first image for preview: Saline_Gastro_0002_00001-12912-04334.tif


## 3. Preview Single Image

Load and visualize a single 2-channel image to verify channels.

## 4. Initialize Cellpose Models

Load two Cellpose models:
- **cyto3** for large cell segmentation (Channel 1)
- **nuclei** or **cyto3** for small protein markers (Channel 2)

In [99]:
# Model for large cells (Channel 1)
model_cells = models.CellposeModel(gpu=use_gpu, model_type='cyto3')
print("✓ Loaded cyto3 model for cell segmentation")

# Model for small protein markers (Channel 2)
# Use 'nuclei' for small round objects or 'cyto3' for general small objects
model_protein = models.CellposeModel(gpu=use_gpu, model_type='nuclei')
print("✓ Loaded nuclei model for protein marker segmentation")

model_type argument is not used in v4.0.1+. Ignoring this argument...
model_type argument is not used in v4.0.1+. Ignoring this argument...


✓ Loaded cyto3 model for cell segmentation
✓ Loaded nuclei model for protein marker segmentation


## 5. Test Segmentation on Single Image

Run segmentation on both channels to verify parameters.

In [100]:
if IMAGE_PATH:
    # Segment Channel 1: Cells (diameter ~250 pixels)
    print("Segmenting Channel 1 (Cell Marker)...")
    masks_cells, flows_cells, styles_cells = model_cells.eval(
        ch_cells,
        diameter=120,           # Expected diameter for cells (200-300 pixels)
        flow_threshold=0.4,
        cellprob_threshold=0.0,
        channels=[0, 0]         # Grayscale input
    )
    num_cells = masks_cells.max()
    print(f"  ✓ Found {num_cells} cells")
    
    # Segment Channel 2: Protein markers (diameter ~15 pixels)
    print("\nSegmenting Channel 2 (Protein Marker)...")
    masks_protein, flows_protein, styles_protein = model_protein.eval(
        ch_protein,
        diameter=6,            # Expected diameter for protein markers (10-20 pixels)
        flow_threshold=0.5,
        cellprob_threshold=-1,
        channels=[0, 0]         # Grayscale input
    )
    num_proteins = masks_protein.max()
    print(f"  ✓ Found {num_proteins} protein markers (initial)")
    
    # Visualize results with napari
    viewer = napari.Viewer()
    
    # Add original images
    viewer.add_image(ch_cells, name='Ch1: Cell Marker (Original)', colormap='green', blending='additive')
    viewer.add_image(ch_protein, name='Ch2: Protein Marker (Original)', colormap='magenta', blending='additive')
    
    # Add segmentation masks
    viewer.add_labels(masks_cells, name=f'Ch1: Cell Masks ({num_cells} cells)')
    viewer.add_labels(masks_protein, name=f'Ch2: Protein Masks ({num_proteins} markers)')
    
    print(f"\n✓ Napari viewer opened with images and segmentation masks")
    print(f"  - Toggle layers on/off using the eye icons")
    print(f"  - Adjust opacity and colormap in the layer controls")
    print(f"  - Run the next cell to apply size filtering to protein masks")
else:
    print("No image to segment!")

channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used


Segmenting Channel 1 (Cell Marker)...


Resizing is deprecated in v4.0.1+
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used


  ✓ Found 346 cells

Segmenting Channel 2 (Protein Marker)...


Resizing is deprecated in v4.0.1+


  ✓ Found 1498 protein markers (initial)

✓ Napari viewer opened with images and segmentation masks
  - Toggle layers on/off using the eye icons
  - Adjust opacity and colormap in the layer controls
  - Run the next cell to apply size filtering to protein masks


## 5.5 Filter Protein Masks by Size

Calculate average protein marker size and filter out masks that are too large (>2x average) or too small (<0.5x average).

In [101]:
def filter_masks_by_size(masks, min_ratio=0.5, max_ratio=2.0):
    """
    Filter segmentation masks based on size relative to average.
    
    Parameters:
    -----------
    masks : numpy.ndarray
        Label mask array
    min_ratio : float
        Minimum size as ratio of average (default: 0.5 = 50% of average)
    max_ratio : float
        Maximum size as ratio of average (default: 2.0 = 200% of average)
    
    Returns:
    --------
    filtered_masks : numpy.ndarray
        Filtered label mask with outliers removed
    stats : dict
        Statistics about filtering
    """
    # Get all unique labels (excluding background 0)
    unique_labels = np.unique(masks)
    unique_labels = unique_labels[unique_labels != 0]
    
    if len(unique_labels) == 0:
        return masks, {'original_count': 0, 'filtered_count': 0, 'avg_size': 0}
    
    # Calculate size of each mask
    sizes = []
    for label_id in unique_labels:
        size = np.sum(masks == label_id)
        sizes.append(size)
    
    sizes = np.array(sizes)
    avg_size = np.mean(sizes)
    
    # Calculate size thresholds
    min_size = avg_size * min_ratio
    max_size = avg_size * max_ratio
    
    print(f"  Mask size statistics:")
    print(f"    Average size: {avg_size:.1f} pixels")
    print(f"    Size range: [{sizes.min()}, {sizes.max()}]")
    print(f"    Filter thresholds: [{min_size:.1f}, {max_size:.1f}]")
    
    # Create filtered mask
    filtered_masks = np.zeros_like(masks)
    new_label = 1
    removed_count = 0
    
    for label_id, size in zip(unique_labels, sizes):
        if min_size <= size <= max_size:
            # Keep this mask with new sequential label
            filtered_masks[masks == label_id] = new_label
            new_label += 1
        else:
            removed_count += 1
    
    stats = {
        'original_count': len(unique_labels),
        'filtered_count': new_label - 1,
        'removed_count': removed_count,
        'avg_size': avg_size,
        'min_threshold': min_size,
        'max_threshold': max_size
    }
    
    print(f"    Removed {removed_count} masks (kept {stats['filtered_count']}/{len(unique_labels)})")
    
    return filtered_masks, stats


# Apply filtering to protein masks from the test segmentation
if 'masks_protein' in locals() and masks_protein is not None:
    print("Filtering protein masks by size...")
    masks_protein_filtered, filter_stats = filter_masks_by_size(
        masks_protein, 
        min_ratio=0.5,  # Remove masks smaller than 50% of average
        max_ratio=2.0   # Remove masks larger than 200% of average
    )
    
    # Visualize filtered results
    viewer = napari.Viewer()
    viewer.add_image(ch_protein, name='Ch2: Protein Marker', colormap='magenta', blending='additive')
    viewer.add_labels(masks_protein, name=f'Original ({filter_stats["original_count"]} markers)')
    viewer.add_labels(masks_protein_filtered, name=f'Filtered ({filter_stats["filtered_count"]} markers)')
    
    print(f"\n✓ Napari viewer opened - compare original vs filtered masks")
else:
    print("No protein masks found. Run test segmentation first (Cell 9).")

Filtering protein masks by size...
  Mask size statistics:
    Average size: 258.4 pixels
    Size range: [26, 842]
    Filter thresholds: [129.2, 516.8]
    Removed 249 masks (kept 1249/1498)

✓ Napari viewer opened - compare original vs filtered masks


## 5.6 Background Subtraction for Channel 1

Prepare Channel 1 for protein segmentation:
1. Use cell mask to calculate background intensity (from regions outside cells)
2. Subtract background from entire Channel 1 image
3. Apply Gaussian blur to reduce noise

This produces a clean image ready for segmentation in the next step.

In [102]:
from skimage import filters

# Apply to test image if available
if 'ch_cells' in locals() and 'masks_cells' in locals() and ch_cells is not None and masks_cells is not None:
    print("Processing Channel 1: Background Subtraction + Blur")
    print("-" * 60)
    
    # Step 1: Calculate background intensity using cell masks
    # Background is the area outside cells (where mask = 0)
    background_mask = (masks_cells == 0)
    
    # Calculate mean background intensity from regions outside cells
    background_intensity = np.mean(ch_cells[background_mask]) + 500
    print(f"  ✓ Calculated background intensity: {background_intensity:.1f}")
    
    # Step 2: Subtract background from entire image
    ch_cells_bg_subtracted = ch_cells.astype(np.float32) - background_intensity
    ch_cells_bg_subtracted = np.clip(ch_cells_bg_subtracted, 0, None)  # Remove negative values
    print(f"  ✓ Background subtracted from Channel 1")
    print(f"    Range after subtraction: [{ch_cells_bg_subtracted.min():.1f}, {ch_cells_bg_subtracted.max():.1f}]")
    
    # Step 3: Apply Gaussian blur (adjust sigma to tune smoothness)
    sigma = 1  # <-- Adjust this value (typically 1-3)
    ch_cells_processed = filters.gaussian(ch_cells_bg_subtracted, sigma=sigma, preserve_range=True)
    ch_cells_processed = ch_cells_processed.astype(np.uint16)
else:
    print("No Channel 1 data or cell masks found. Run test segmentation first (cells 5-9).")

Processing Channel 1: Background Subtraction + Blur
------------------------------------------------------------
  ✓ Calculated background intensity: 706.3
  ✓ Background subtracted from Channel 1
    Range after subtraction: [0.0, 32802.7]


In [103]:
from skimage.feature import blob_log

def detect_proteins_blob(ch_cells, masks_cells, sigma=1, min_sigma=0.8, max_sigma=2.0, 
                         blob_threshold=0.01, overlap=0.5, background_offset=500,
                         min_circularity=0.5):
    """
    Detect small proteins in Channel 1 using blob detection.
    
    Steps:
    1. Calculate background from cell-free regions
    2. Subtract background
    3. Apply Gaussian blur
    4. Detect blobs using Laplacian of Gaussian
    5. Convert blobs to segmentation masks
    6. Filter by shape (remove elongated lines)
    
    Parameters:
    -----------
    ch_cells : numpy.ndarray
        Raw Channel 1 image
    masks_cells : numpy.ndarray
        Cell segmentation masks (to calculate background)
    sigma : float
        Gaussian blur sigma (default: 1)
    min_sigma : float
        Minimum blob size for detection (default: 0.8)
    max_sigma : float
        Maximum blob size for detection (default: 2.0)
    blob_threshold : float
        Detection sensitivity, lower = more blobs (default: 0.01)
    overlap : float
        Maximum allowed overlap between blobs (default: 0.5)
    background_offset : float
        Additional offset added to background intensity (default: 500)
    min_circularity : float
        Minimum circularity to keep (0.5-0.7, default: 0.5)
        Higher = only very round objects
    
    Returns:
    --------
    masks : numpy.ndarray
        Segmentation masks for detected proteins (shape-filtered)
    processed_img : numpy.ndarray
        Background-subtracted and blurred image
    """
    from skimage import filters
    
    # Step 1: Calculate background
    background_mask = (masks_cells == 0)
    background_intensity = np.mean(ch_cells[background_mask]) + background_offset
    
    # Step 2: Subtract background
    img_bg_subtracted = ch_cells.astype(np.float32) - background_intensity
    img_bg_subtracted = np.clip(img_bg_subtracted, 0, None)
    
    # Step 3: Apply Gaussian blur
    img_processed = filters.gaussian(img_bg_subtracted, sigma=sigma, preserve_range=True)
    img_processed = img_processed.astype(np.uint16)
    
    # Step 4: Normalize for blob detection
    img_norm = img_processed.astype(np.float32)
    if img_norm.max() > 0:
        img_norm = img_norm / img_norm.max()
    
    # Step 5: Detect blobs
    blobs = blob_log(img_norm, min_sigma=min_sigma, max_sigma=max_sigma, 
                     threshold=blob_threshold, overlap=overlap)
    
    # Step 6: Convert blobs to masks
    masks_raw = np.zeros(ch_cells.shape, dtype=np.uint16)
    for idx, blob in enumerate(blobs, 1):
        y, x, sigma_detected = blob
        radius = int(sigma_detected * np.sqrt(2))
        
        # Create circular mask
        yy, xx = np.ogrid[:ch_cells.shape[0], :ch_cells.shape[1]]
        circle_mask = (xx - x)**2 + (yy - y)**2 <= radius**2
        masks_raw[circle_mask] = idx
    
    # Step 7: Filter by shape to remove elongated lines (if any masks detected)
    if masks_raw.max() > 0 and min_circularity > 0:
        from skimage.measure import regionprops
        
        regions = regionprops(masks_raw)
        masks = np.zeros_like(masks_raw)
        new_label = 1
        
        for region in regions:
            area = region.area
            perimeter = region.perimeter
            
            if perimeter > 0:
                circularity = 4 * np.pi * area / (perimeter ** 2)
            else:
                circularity = 0
            
            # Keep only round objects
            if circularity >= min_circularity:
                masks[masks_raw == region.label] = new_label
                new_label += 1
    else:
        masks = masks_raw
    
    return masks, img_processed


# Test the function with current data
if 'ch_cells' in locals() and 'masks_cells' in locals():
    print("Testing blob detection function...")
    
    # Use the parameters from the previous cell
    test_masks, test_processed = detect_proteins_blob(
        ch_cells, 
        masks_cells,
        sigma=1,
        min_sigma=0.5,
        max_sigma=2.0,
        blob_threshold=0.015,
        overlap=0.001,
        background_offset=400,
        min_circularity=0.6  # Remove elongated lines, keep round blobs
    )
    
    print(f"✓ Function test successful!")
    print(f"  - Found {test_masks.max()} proteins")
    print(f"  - Processed image shape: {test_processed.shape}")
    
    # Apply size filtering
    test_masks_filtered, test_stats = filter_masks_by_size(test_masks, min_ratio=0.5, max_ratio=2.0)
    
    # Visualize with napari
    viewer = napari.Viewer()
    viewer.add_image(ch_cells, name='Ch1: Original', colormap='green', blending='additive', visible=False)
    viewer.add_labels(masks_cells, name='Cell Masks', opacity=0.3, visible=False)
    viewer.add_image(test_processed, name='Ch1: Background Subtracted', colormap='green', blending='additive')
    viewer.add_labels(test_masks, name=f'Raw Detections ({test_stats["original_count"]})', opacity=0.5)
    viewer.add_labels(test_masks_filtered, name=f'Filtered Proteins ({test_stats["filtered_count"]})')
    
    print(f"  - Napari viewer opened to verify results")
    print(f"  - Ready for batch processing!")
else:
    print("Run cells 5-9 first to test the function.")


Testing blob detection function...
✓ Function test successful!
  - Found 220 proteins
  - Processed image shape: (3520, 3520)
  Mask size statistics:
    Average size: 12.8 pixels
    Size range: [5, 13]
    Filter thresholds: [6.4, 25.6]
    Removed 5 masks (kept 215/220)
  - Napari viewer opened to verify results
  - Ready for batch processing!


## 5.8.5 Filter by Shape (Remove Elongated Lines)

Remove elongated/line-shaped objects and keep only round blobs by filtering based on circularity.

## 5.8 Create Blob Detection Function for Batch Processing

Define a reusable function that applies blob detection with shape filtering to remove elongated lines and keep only round blobs.

## 5.7 Segment Channel 1 Proteins (Blob Detection)

**Blob Detection (Laplacian of Gaussian)** - Optimized for detecting small round objects (9-10 pixel diameter).

**Parameter Guide:**
- **min_sigma / max_sigma**: Control the size range of detected blobs
  - For 9-10px diameter objects, try min_sigma=0.8-1.2, max_sigma=1.5-2.5
  - Increase if blobs are too small, decrease if too large
- **blob_threshold**: Controls detection sensitivity
  - Lower values (0.005-0.01) = more sensitive, detects more blobs
  - Higher values (0.02-0.05) = less sensitive, only strong blobs
- **overlap**: How much blobs can overlap (0.5 = allows 50% overlap)

**Note:** After blob detection, run Section 5.8.5 to filter by shape and remove elongated lines.

## 6. Batch Processing

Process all TIF files in the folder and save:
- **Cell masks (Channel 1):** Large cell segmentation using Cellpose
- **Ch1 Protein masks:** Small proteins detected using **Blob Detection (LoG)** with **shape filtering** to remove elongated lines
- **Ch2 Protein masks:** Small proteins from Channel 2 using Cellpose **with size filtering applied**
- Overlay images with boundaries for all three segmentation types

**Note:** 
- Ch1 uses optimized blob detection (Laplacian of Gaussian) for small round objects
- **Shape filtering** removes elongated lines/artifacts (only keeps circular objects with circularity >= 0.5)
- Background is calculated from regions outside cells and subtracted
- All protein masks are filtered to remove outliers (>2x or <0.5x average size)

## 7. Export Summary Statistics (Optional)

Save quantification results to CSV for further analysis.

In [ ]:
import pandas as pd

if 'results_summary' in locals() and results_summary:
    # Create DataFrame
    df = pd.DataFrame(results_summary)
    
    # Save to CSV
    csv_path = Path(FOLDER_PATH) / 'IF_segmentation_summary.csv'
    df.to_csv(csv_path, index=False)
    
    print(f"✓ Summary saved to: {csv_path}")
    print("\nPreview:")
    display(df)
else:
    print("No results to save. Run batch processing first.")